# Przetwarzanie Danych Ustrukturyzowanych. Praca domowa nr 4.

**Maciej Borkowski**,
**15 czerwca 2022r.**


Projekt zrealizowany w ramach przedmiotu *Przetwarzanie Danych Ustrukturyzowanych, Inżynieria i Analiza Danych sem. letni 2021/2022.*

Zadaniem projektowym jest rozwiązanie pięciu zadań zapisanych w postaci poleceń SQL przy użyciu wywołań funkcji i metod z pakietu `pandas`. Rozwiązanie każdego zadania zawiera ponadto komentarze oraz sprawdzenie równoważności z poleceniem SQL.

In [1]:
import pandas as pd
import numpy as np
import os, os.path
import sqlite3

In [2]:
Badges = pd.read_csv("pliki/Badges.csv.gz", compression = 'gzip')
Comments = pd.read_csv("pliki/Comments.csv.gz", compression = 'gzip')
PostLinks = pd.read_csv("pliki/PostLinks.csv.gz", compression = 'gzip')
Posts = pd.read_csv("pliki/Posts.csv.gz", compression = 'gzip')
Tags = pd.read_csv("pliki/Tags.csv.gz", compression = 'gzip')
Users = pd.read_csv("pliki/Users.csv.gz", compression = 'gzip')
Votes = pd.read_csv("pliki/Votes.csv.gz", compression = 'gzip')

In [3]:
baza = os.path.join('baza_danych.db')
if os.path.isfile(baza): 
    os.remove(baza)
    
conn = sqlite3.connect(baza)
    
Badges.to_sql("Badges", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

953600

### Zadanie 1

In [4]:
df_sql_1 = pd.read_sql_query("""
SELECT Count, TagName 
FROM Tags 
WHERE Count > 1000 ORDER BY Count DESC
""", conn)
df_sql_1

,Count,TagName
0,9470,visas
1,5119,usa
2,4601,uk
3,4460,air-travel
4,3503,customs-and-immigration
5,3296,schengen
6,2058,transit
7,1695,passports
8,1665,indian-citizens
9,1517,trains


In [5]:
# wybieramy kolumny Count i TagName tych obserwacji, dla których wartość Count jest większa od 1000, następnie
# sortujemy w porządku malejącym po wartości Count i porządkujemy wiersze

df_base_1 = Tags.loc[
    Tags.Count > 1000, ["Count", "TagName"]]\
    .sort_values("Count", ascending = False)\
    .reset_index(drop=True)    

In [6]:
# Sprawdzenie równoważności rozwiązania
df_sql_1.equals(df_base_1)

True

### Zadanie 2

In [7]:
df_sql_2 = pd.read_sql_query("""
SELECT Location, COUNT(*) AS Count 
FROM (
    SELECT Posts.OwnerUserId, Users.Id, Users.Location 
    FROM Users
    JOIN Posts ON Users.Id = Posts.OwnerUserId
)
WHERE Location NOT IN ('') 
GROUP BY Location
ORDER BY Count DESC
LIMIT 10""", conn)
df_sql_2

,Location,Count
0,"Christchurch, New Zealand",2765
1,"New York, NY",1788
2,"London, United Kingdom",1708
3,UK,1590
4,"Sunshine Coast QLD, Australia",1550
5,Australia,1183
6,"Vancouver, Canada",967
7,Netherlands,935
8,on the server farm,924
9,Pennsylvania,921


In [8]:
# łączymy ramki Users i Posts

x = pd.merge(Users, Posts, left_on="Id", right_on="OwnerUserId")

# wybieramy obserwacje, dla których wartość Location jest określona,
# zliczamy obserwacje dla każdej z wartości Location, a następnie sortujemy malejąco względem ilości tych obserwacji,
# wybieramy pierwsze 10 obserwacji

df_base_2 = x.loc[x.Location != ""].Location.value_counts()\
    .reset_index().rename(columns = {"index":"Location", "Location":"Count"})\
    .sort_values("Count", ascending = False).head(10)

In [9]:
# Sprawdzenie równoważności rozwiązania
df_sql_2.equals(df_base_2)

True

### Zadanie 3

In [10]:
df_sql_3 = pd.read_sql_query("""
SELECT Year, SUM(Number) AS TotalNumber 
FROM (
    SELECT 
        Name, 
        COUNT(*) AS Number, 
        STRFTIME('%Y', Badges.Date) AS Year
    FROM Badges
    WHERE Class = 1 
    GROUP BY Name, Year
)
GROUP BY Year
ORDER BY TotalNumber
""", conn)
df_sql_3

,Year,TotalNumber
0,2011,16
1,2012,23
2,2013,66
3,2021,153
4,2014,197
5,2020,265
6,2015,328
7,2016,509
8,2017,552
9,2018,697


In [11]:
# wybieramy obserwacje, dla których wartość zmiennej Class jest równa 1,
# zliczamy ilość obserwacji w grupach będących kombinacją wartości Name i roku wyciągniętego ze zmiennej Date,
# sumujemy otrzymane liczności dla każdego roku,
# sortujemy po zmiennej TotalNumber i porządkujemy indeksy

df_base_3 = Badges.loc[Badges.Class == 1, :].groupby(["Name", Badges["Date"].str[:4]]) \
    .size().reset_index().rename(columns = {"Date":"Year", 0: "Number"})\
    .groupby("Year").Number.agg(np.sum).reset_index()\
    .rename(columns = {"Number":"TotalNumber"}).sort_values("TotalNumber")\
    .reset_index(drop=True)

In [12]:
# Sprawdzenie równoważności rozwiązania
df_sql_3.equals(df_base_3)

True

### Zadanie 4

In [13]:
df_sql_4 = pd.read_sql_query("""
SELECT
    Users.AccountId,
    Users.DisplayName,
    Users.Location,
    AVG(PostAuth.AnswersCount) as AverageAnswersCount
FROM
(
    SELECT AnsCount.AnswersCount, Posts.Id, Posts.OwnerUserId
    FROM (
        SELECT Posts.ParentId, COUNT(*) AS AnswersCount 
        FROM Posts
        WHERE Posts.PostTypeId = 2
        GROUP BY Posts.ParentId
        ) AS AnsCount
    JOIN Posts ON Posts.Id = AnsCount.ParentId
) AS PostAuth
JOIN Users ON Users.AccountId=PostAuth.OwnerUserId 
GROUP BY OwnerUserId
ORDER BY AverageAnswersCount DESC, AccountId ASC 
LIMIT 10
""", conn)
df_sql_4

,AccountId,DisplayName,Location,AverageAnswersCount
0,280.0,csmba,"San Francisco, CA",11.0
1,40811.0,vocaro,"San Jose, CA",11.0
2,204.0,Josh,Australia,10.0
3,44093.0,Emma Arbogast,"Salem, OR",10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,19588.0,JD Isaacks,"Atlanta, GA",8.0
6,20473.0,Jeremy Boyd,"Houston, TX",8.0
7,42364.0,Petrogad,None,8.0
8,54571.0,Christian,None,8.0
9,79346.0,Thomas Matthews,California,8.0


In [14]:
# tworzymy ramkę AnsCount - wybieramy obserwacje z ramki Posts, dla których wartość zmiennej 
# PostTypeId jest równa 2, grupujemy obserwacje zmienną ParentId i zliczamy obserwacje w grupach

AnsCount = Posts.loc[Posts.PostTypeId == 2, :].groupby("ParentId").size().reset_index()\
            .rename(columns = {0:"AnswersCount"})

# tworzymy ramkę PostAuth - łączymy ramki Posts i AnsCount,
# grupujemy obserwacje zmienną OwnerUserId i liczymy średnią wartość zmiennej AnswersCount dla obserwacji w grupach

PostAuth = pd.merge(AnsCount, Posts, left_on="ParentId", right_on="Id")\
    .groupby("OwnerUserId").AnswersCount.mean().reset_index()\
    .rename(columns = {"AnswersCount":"AverageAnswersCount"})

# łączymy ramki Users i PostAuth, wybieramy zadane kolumny, 
# sortujemy w odpowiedniej kolejności malejąco po zmiennej AverageAnswersCount i rosnąco po zmiennej AccountId,
# wybieramy pierwsze 10 obserwacji

df_base_4 = pd.merge(Users, PostAuth, left_on="AccountId", right_on="OwnerUserId")\
    .loc[:,["AccountId", "DisplayName", "Location", "AverageAnswersCount"]]\
    .sort_values(["AverageAnswersCount","AccountId"], ascending = [False, True])\
    .reset_index(drop=True).head(10)   

In [15]:
# Sprawdzenie równoważności rozwiązania
df_sql_4.equals(df_base_4)

True

### Zadanie 5

In [16]:
df_sql_5 = pd.read_sql_query("""
SELECT 
    Posts.Title, 
    Posts.Id,
    STRFTIME('%Y-%m-%d', Posts.CreationDate) AS Date, 
    VotesByAge.Votes
FROM Posts 
JOIN (
    SELECT
        PostId,
        MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes, 
        MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes, 
        SUM(Total) AS Votes
    FROM ( 
        SELECT
            PostId,
            CASE STRFTIME('%Y', CreationDate)
                WHEN '2021' THEN 'new' 
                WHEN '2020' THEN 'new' 
                ELSE 'old'
                END VoteDate,
            COUNT(*) AS Total 
        FROM Votes
        WHERE VoteTypeId IN (1, 2, 5)
        GROUP BY PostId, VoteDate 
    ) AS VotesDates
    GROUP BY VotesDates.PostId
    HAVING NewVotes > OldVotes
) AS VotesByAge ON Posts.Id = VotesByAge.PostId 
WHERE Title NOT IN ('')
ORDER BY Votes DESC
LIMIT 10
""", conn)
df_sql_5

,Title,Id,Date,Votes
0,What's the longest distance that can be travel...,151994,2020-01-09,140
1,Considerations for very fragile and expensive ...,157138,2020-05-10,108
2,What are these chair-like things in hotels?,153605,2020-02-10,94
3,What to do if I overstayed my e-visa for Saint...,152418,2020-01-18,84
4,What are the hidden fees of a cruise?,152178,2020-01-13,80
5,What happens when a town is under quarantine a...,152677,2020-01-23,76
6,I've been warned to leave the US within 10 day...,156945,2020-04-28,72
7,"When an individual enters the United States, c...",159298,2020-09-06,68
8,Is Seiryu Miharashi Station the only train sta...,162386,2021-02-04,64
9,Positive drug test in Singapore: How long do I...,163264,2021-04-08,64


In [17]:
# tworzymy ramkę VotesDates - z ramki Votes wybieramy obserwacje, które przyjmują wartość 
# 1, 2 lub 5 dla zmiennej VoteTypeId, tworzymy kolumnę czynników VoteDate - dla roku 
# wyciągniętego z zmiennej CreationDate przypisujemy "new", jeśli rok to 2021 lub 2020, lub "old" w przeciwnym wypadku
# grupujemy obserwacje zmiennymi PostId i VoteDate i zliczamy obserwacje w grupach

VotesDates = Votes.loc[Votes.VoteTypeId.isin([1, 2, 5])].reset_index()
VotesDates = VotesDates.assign(
    VoteDate = np.where( VotesDates["CreationDate"].str[:4].isin(["2021", "2020"]),"new","old"))\
    .groupby(["PostId", "VoteDate"]).size().reset_index().rename(columns = {0:"Total"})


# tworzymy ramkę VotesByAge - grupujemy zmienną PostId i tworzymy kolumny NewVotes, OldVotes oraz Votes,
# wybieramy te obserwacje, dla których wartość zmiennej NewVotes jest większa od OldVotes,

VotesByAge = VotesDates.groupby("PostId").apply(
    lambda x: pd.Series([
                        max(np.where(x.VoteDate == "new", x.Total, 0)), 
                        max(np.where(x.VoteDate == "old", x.Total, 0)),
                        sum(x.Total)],
                        index=["NewVotes", "OldVotes", "Votes"])).reset_index()
VotesByAge = VotesByAge[VotesByAge.NewVotes > VotesByAge.OldVotes].reset_index()

# łączymy ramki Posts i VotesByAge, wyciągamy datę ze zmiennej CreationDate, 
# wybieramy obserwacje dla których wartość zmiennej Title jest określona, wybieramy zadane kolumny, 
# sortujemy malejąco zmienną Votes oraz wybieramy pierwsze 10 obserwacji

df_base_5 = pd.merge(Posts, VotesByAge, left_on="Id", right_on="PostId")
df_base_5 = df_base_5.assign(Date = df_base_5["CreationDate"].str[:10])\
        .loc[~df_base_5["Title"].isnull(), ["Title", "Id", "Date", "Votes"]]\
        .sort_values("Votes", ascending = False).reset_index(drop=True).head(10)

In [18]:
# Sprawdzenie równoważności rozwiązania
df_sql_5.equals(df_base_5)

False

In [19]:
df_base_5

,Title,Id,Date,Votes
0,What's the longest distance that can be travel...,151994,2020-01-09,140
1,Considerations for very fragile and expensive ...,157138,2020-05-10,108
2,What are these chair-like things in hotels?,153605,2020-02-10,94
3,What to do if I overstayed my e-visa for Saint...,152418,2020-01-18,84
4,What are the hidden fees of a cruise?,152178,2020-01-13,80
5,What happens when a town is under quarantine a...,152677,2020-01-23,76
6,I've been warned to leave the US within 10 day...,156945,2020-04-28,72
7,"When an individual enters the United States, c...",159298,2020-09-06,68
8,Positive drug test in Singapore: How long do I...,163264,2021-04-08,64
9,Is Seiryu Miharashi Station the only train sta...,162386,2021-02-04,64


In [20]:
# Widzimy że obserwacje 8 i 9 są w odwrotnej kolejności względem rozwiązania sql, może być to spowodowane różnicom
# w algorytmie sortowania, ponieważ wartość zmiennej Votes jest dla tych obserwacji równa